<a href="https://colab.research.google.com/github/kyochanpy/forest_bears/blob/main/make_df/03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import datetime as dt
import re

# defs

In [16]:
# columnsを設定
def set_columns(input_df):
    input_df_ = input_df.copy()

    output_df = input_df_.shift()                
    output_df.iloc[0] = output_df.columns.values
    output_df.columns = ['No.', '区', '発生日時', '出没場所', '状況']

    return output_df

In [17]:
# 崩れた部分を修正し、時間を文字列に
def fix_df(input_df):
    input_df_ = input_df.copy()
    
    idx_list = []
    no_list = input_df_["No."].tolist()
    for i in no_list:
        if np.isnan(i):
            idx_list.append(no_list.index(i))

    for i in idx_list:
        i_ = input_df_.iloc[i, 2]
        if type(i_) != float:
            input_df_.iloc[i, 2]  = i_.strftime('%H:%M')

    input_df_ = input_df_.fillna("")

    for i in idx_list:
        input_df_.iloc[i-1, 2:4] = input_df_.iloc[i-1, 2:4] + input_df_.iloc[i, 2:4]
        
    output_df = input_df_.drop(idx_list)

    return output_df

In [18]:
# 上二つの関数を適用
def make_df():
    output_df = pd.DataFrame()
    for i in range(5):
        excel_df = pd.read_excel("/content/drive/MyDrive/forest_bears/forest_bears.xlsx", sheet_name=i)
        tmp_df = set_columns(excel_df)
        df = fix_df(tmp_df)
        output_df = pd.concat([output_df, df])
    
    return output_df

In [47]:
# 年月日・時間についての特徴量生成
def get_date_time_features(input_df):
    input_df_ = input_df.copy()

    no_values = input_df_["No."].values
    len_list_ = []
    for i in range(len(no_values)+1):
        if no_values[i+1] - no_values[i] != 1:
            len_list_.append(i)
            if len(len_list_) == 4:
                break
    len_list_.append(len(no_values))
    
    len_list = []
    for i in range(0,len(len_list_)):
        i_ = len_list_[i] - len_list_[i-1]
        len_list.append(i_)
    len_list[0] = len_list_[0]

    fiscal_year_list = []
    for i, j in zip(
        len_list,
        ["h28", "h29", "h30", "r1", "r2"]
    ):
        fiscal_year_list = fiscal_year_list + [j] * i
    
    year_list = []
    month_list = []
    day_list = []
    day_week_list = []
    time_list = []
    for i in input_df_["発生日時"].values:
        i = i.replace(" ", "")
        if len(i) > 14:
            i_list = re.findall(r"\d+", i)
            if i_list[0] == "28":
                year_list.append(f"h{i_list[0]}")
            else:
                year_list.append(f"r{i_list[0]}")
            month_list.append(i_list[1])
            day_list.append(i_list[2])
            day_week_list.append(i[-7])
            time_list.append(i[-5:])
        elif len(i) <= 10:
            i_list = re.findall(r"\d+", i)
            year_list.append("不明")
            month_list.append(i_list[0])
            day_list.append(i_list[1])
            day_week_list.append(i[-2])
            time_list.append("不明")
        elif (len(i) > 10) and (i[-3] == ":"):
            i_list = re.findall(r"\d+", i)
            year_list.append("不明")
            month_list.append(i_list[0])
            day_list.append(i_list[1])
            day_week_list.append(i[-7])
            time_list.append(i[-5:])
        else:
            i_list = re.findall(r"\d+", i)
            if i_list[0] == "28":
                year_list.append(f"h{i_list[0]}")
            else:
                year_list.append(f"r{i_list[0]}")
            month_list.append(i_list[1])
            day_list.append(i_list[2])
            day_week_list.append(i[-2])
            time_list.append("不明")

    output_df = pd.DataFrame()
    output_df["年度"] = fiscal_year_list
    output_df["年"] = year_list
    output_df["月"] = month_list
    output_df["日"] = day_list
    output_df["曜日"] = day_week_list
    output_df["時間"] = time_list
   
    fixed_year_list = []
    for f, y, m in zip(
        output_df["年度"].values,
        output_df["年"].values,
        output_df["月"].values
    ):
        if y != "不明":
            fixed_year_list.append(y)
        else:
            if 4 <= int(m) <= 12:
                fixed_year_list.append(f)
            else:
                if y == "r2":
                    fixed_year_list.append("r3")
                else:
                    fixed_year_list.append(fixed_year_list[fixed_year_list.index(y)+1])
    output_df["年"] = fixed_year_list  

    return output_df

In [35]:
df = make_df()

In [36]:
df.to_csv("forest_bears_base_df.csv", index=False)

In [31]:
df

,No.,区,発生日時,出没場所,状況
0,1,南区,平28年 4月14日(木),南区南沢市民の森 案内板2番と3番の間,足跡を確認
2,2,南区,平28年 4月20日(水),南区白川市民の森 案内板7番から8番へ30m地点 案内板5番から21番へ200m地点,足跡・爪痕を確認
4,3,南区,平28年 4月21日(木),南区定山渓920番地付近 豊羽鉱山への道道,足跡を確認
5,4,南区,平28年 4月28日(木),南区真駒内柏丘1丁目 真駒内川河川敷,ヒグマらしき動物を目撃
6,5,南区,平28年 5月14日(土),南区定山渓（定山渓自然の村南東約300mの林道）,ヒグマを目撃
...,...,...,...,...,...
142,91,手稲区,令和2年11月12日(木),手稲区手稲本町593番地151付近（手稲山麓西線上）,足跡・鳴き声を確認
143,92,西区,令和2年12月4日(金),西区小別沢20番地付近,足跡、被毛を確認
144,93,南区,令和2年12月23日(水)18:30,南区石山1045番地2付近,ヒグマを目撃
146,94,南区,令和3年3月25日(木),南区白川市民の森(案内板20番から17番方向へ約20mの付近),足跡を確認


In [32]:
date_time_df = get_date_time_features(df)

In [33]:
date_time_df

,年度,年,月,日,曜日,時間
0,h28,h28,4,14,木,不明
1,h28,h28,4,20,水,不明
2,h28,h28,4,21,木,不明
3,h28,h28,4,28,木,不明
4,h28,h28,5,14,土,不明
...,...,...,...,...,...,...
558,r2,r2,11,12,木,不明
559,r2,r2,12,4,金,不明
560,r2,r2,12,23,水,18:30
561,r2,r3,3,25,木,不明


In [34]:
date_time_df.to_csv("forest_bears_date_time_df.csv", index=False)

In [37]:
address_df = pd.read_csv("/content/drive/MyDrive/forest_bears/forest_bears_address_df.csv")

In [40]:
df = pd.concat([address_df, date_time_df], axis=1)

In [45]:
df.to_csv("forest_bears_data_time_address_df.csv", index=False)

In [46]:
df

,No.,区,発生日時,出没場所,状況,lat,lng,年度,年,月,日,曜日,時間
0,1.0,南区,平28年 4月14日(木),南区南沢市民の森 案内板2番と3番の間,足跡を確認,42.9841135,141.3019655,h28,h28,4,14,木,不明
1,2.0,南区,平28年 4月20日(水),南区白川市民の森 案内板7番から8番へ30m地点 案内板5番から21番へ200m地点,足跡・爪痕を確認,42.983615,141.2832357,h28,h28,4,20,水,不明
2,3.0,南区,平28年 4月21日(木),南区定山渓920番地付近 豊羽鉱山への道道,足跡を確認,42.982008,141.0418291,h28,h28,4,21,木,不明
3,4.0,南区,平28年 4月28日(木),南区真駒内柏丘1丁目 真駒内川河川敷,ヒグマらしき動物を目撃,42.9843856,141.3487511,h28,h28,4,28,木,不明
4,5.0,南区,平28年 5月14日(土),南区定山渓（定山渓自然の村南東約300mの林道）,ヒグマを目撃,42.9313821,141.1520642,h28,h28,5,14,土,不明
...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,91.0,手稲区,令和2年11月12日(木),手稲区手稲本町593番地151付近（手稲山麓西線上）,足跡・鳴き声を確認,43.1103641,141.2200076,r2,r2,11,12,木,不明
559,92.0,西区,令和2年12月4日(金),西区小別沢20番地付近,足跡、被毛を確認,43.0473591,141.2777383,r2,r2,12,4,金,不明
560,93.0,南区,令和2年12月23日(水)18:30,南区石山1045番地2付近,ヒグマを目撃,42.9461915,141.3285571,r2,r2,12,23,水,18:30
561,94.0,南区,令和3年3月25日(木),南区白川市民の森(案内板20番から17番方向へ約20mの付近),足跡を確認,42.983615,141.2832357,r2,r3,3,25,木,不明


In [50]:
fig = px.scatter_mapbox(df,

                    # Here, plotly gets, (x,y) coordinates
                    lat="lat",
                    lon="lng",
                    #text='年',

                    #Here, plotly detects color of series
                    color="年",
                    labels="年",

                    zoom=9,
                    center={"lat":42.423576, "lon":141.094132},
                    height=600,
                    width=800)
fig.update_layout(mapbox_style='stamen-terrain')
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.update_layout(title_text="GPS trafic")
fig.show()